In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
df2=pd.read_csv("/content/KVK_Field_sensor1 modified - Soda.csv",parse_dates=['Date Time'])
df2.drop(['Wind direction','Snowfall','Snow depth','Short-wave irradiation'],axis='columns',inplace=True)
temp=df2[['Temperature']]
rel=df2[['Relative Humidity']]
pres=df2[['Pressure']]
wind=df2[['Wind speed']]
rain=df2[['Rainfall']]
df=pd.read_csv("/content/KVK_ORG1.xlsx - feeds.csv",parse_dates=['created_at'])
df['created_at'] = df.created_at.dt.strftime('%Y-%m-%d %H:%M:%S')
df1=df.drop(['Air Temperature (degree Centigrate)','entry_id','Relative Air Humidity (%)','Soil moisture (Resistance-Ohm)'],axis='columns')
merged1=pd.concat([df1,temp,rel,pres,wind,rain],axis='columns')
soil=df[['Soil moisture (Centi Bar)']]
merged2=pd.concat([df2,soil],axis='columns')
merged2

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Date Time,Temperature,Relative Humidity,Pressure,Wind speed,Rainfall,Soil moisture (Centi Bar)
0,2020-01-01 01:00:00,278.08,63.40,989.24,1.24,0.0,34.02
1,2020-01-01 02:00:00,278.29,62.04,989.94,1.00,0.0,34.00
2,2020-01-01 03:00:00,279.06,60.07,990.66,0.88,0.0,34.00
3,2020-01-01 04:00:00,281.46,56.88,991.56,1.20,0.0,34.00
4,2020-01-01 05:00:00,284.02,47.38,992.36,1.73,0.0,34.00
...,...,...,...,...,...,...,...
200172,NaT,NaN,NaN,NaN,NaN,NaN,NaN
200173,NaT,NaN,NaN,NaN,NaN,NaN,NaN
200174,NaT,NaN,NaN,NaN,NaN,NaN,NaN
200175,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
merged1.rename(columns = {'created_at':'Date Time'}, inplace = True)
veg=pd.read_csv("/content/Vegetation indices  modified ee-chart.csv",parse_dates=['Vegitation Index'])
veg.drop(0,axis='rows',inplace=True)
veg.rename(columns = {'Vegitation Index':'Date Time'}, inplace = True)
veg.drop(['NDWI1','CAI','EVI','GNDVI','SAVI','MSI','ARI','DWSI','CVI'], axis = 1,inplace=True)
veg.dropna(inplace=True)
merged1['Soil moisture (Centi Bar)'].fillna('0',inplace=True)
merged1['Temperature'].fillna('0',inplace=True)
merged1['Relative Humidity'].fillna('0',inplace=True)
merged1['Pressure'].fillna('0',inplace=True)
merged1['Wind speed'].fillna('0',inplace=True)
merged1['Rainfall'].fillna('0',inplace=True)
merged1.dropna(axis='rows',inplace=True)
merged2.dropna(axis='rows',inplace=True)
veg

,Date Time,NDVI
1,2020-05-12 12:00:00,0.335
2,2020-08-12 12:00:00,0.342
3,2020-10-12 12:00:00,0.401
4,2020-12-15 12:00:00,0.356
5,2020-12-18 12:00:00,0.458
...,...,...
98,2022-02-13 12:00:00,0.357
99,2022-02-16 12:00:00,0.357
100,2022-02-18 12:00:00,0.377
101,2022-02-21 12:00:00,0.405


In [35]:
merged2=merged2[['Date Time','Soil moisture (Centi Bar)','Temperature','Relative Humidity','Pressure','Wind speed','Rainfall']]
merged1['Date Time'] = pd.to_datetime(merged1['Date Time'])
merged2['Date Time'] = pd.to_datetime(merged2['Date Time'])
final=pd.merge_asof(merged1,merged2.sort_values(['Date Time','Temperature']),left_on='Date Time',right_on='Date Time')
final['TEMPERATURE']= final[['Temperature_y','Temperature_x']].median(axis='columns',skipna=True)
final.drop(['Temperature_y','Temperature_x'],axis='columns',inplace=True)
final['RELATIVE_HUMIDITY']= final[['Relative Humidity_x','Relative Humidity_y']].median(axis='columns',skipna=True)
final.drop(['Relative Humidity_x','Relative Humidity_y'],axis='columns',inplace=True)
final['SOIL_MOISTURE']= final[['Soil moisture (Centi Bar)_x','Soil moisture (Centi Bar)_y']].median(axis='columns',skipna=True)
final.drop(['Soil moisture (Centi Bar)_x','Soil moisture (Centi Bar)_y'],axis='columns',inplace=True)
final['PRESSURE']= final[['Pressure_x','Pressure_y']].median(axis='columns',skipna=True)
final.drop(['Pressure_x','Pressure_y'],axis='columns',inplace=True)
final['WIND_SPEED']= final[['Wind speed_x','Wind speed_y']].median(axis='columns',skipna=True)
final.drop(['Wind speed_x','Wind speed_y'],axis='columns',inplace=True)
final['RAINFALL']= final[['Rainfall_x','Rainfall_y']].median(axis='columns',skipna=True)
final.drop(['Rainfall_x','Rainfall_y'],axis='columns',inplace=True)
final_ds=pd.merge_asof(final,veg.sort_values(['Date Time']),left_on='Date Time',right_on='Date Time')
final_ds.dropna(axis='rows',inplace=True)
final_ds

,Date Time,TEMPERATURE,RELATIVE_HUMIDITY,SOIL_MOISTURE,PRESSURE,WIND_SPEED,RAINFALL,NDVI
0,2022-01-25 20:29:44,283.110,63.550,25.970,986.600,1.845,0.009843,0.232
1,2022-01-25 20:30:02,283.215,62.870,25.960,986.950,1.725,0.009843,0.232
2,2022-01-25 20:30:19,283.600,61.885,25.960,987.310,1.665,0.009843,0.232
3,2022-01-25 20:30:37,284.800,60.290,25.960,987.760,1.825,0.009843,0.232
4,2022-01-25 20:30:55,286.080,55.540,25.960,988.160,2.090,0.009843,0.232
...,...,...,...,...,...,...,...,...
200172,2022-03-16 12:40:28,145.720,18.970,8.935,492.415,1.435,0.000000,0.363
200173,2022-03-16 12:40:46,145.720,18.970,8.935,492.415,1.435,0.000000,0.363
200174,2022-03-16 12:41:04,145.720,18.970,8.935,492.415,1.435,0.000000,0.363
200175,2022-03-16 12:41:21,145.720,18.970,8.935,492.415,1.435,0.000000,0.363


In [36]:
final_ds.to_csv("/content/final_ds.csv",index=False)